In [1]:
import numpy as np
import pandas as pd
import feather
from pandasql import sqldf
from fisher import pvalue

In [2]:
# first gene is causal, extreme situation include [1,1,1] and [0,0,0]
s1 = [0,0,0]
s2 = [1,0,0] # need to add a constance in config
s3 = [0,1,0]
s4 = [0,0,1]
s5 = [1,1,0] # need to add a constance in config
s6 = [0,1,1]
s7 = [1,1,1] # need to add a constance in config

In [3]:
def sample(n1, n2, n3, n4, n5, n6, n7, const7, const2, const5):
    overlap = np.matrix( [s7 for i in range(n7)] + [s1 for i in range(n1)] + [s2 for i in range(n2)] 
                       + [s3 for i in range(n3)] + [s4 for i in range(n4)] + [s5 for i in range(n5)] 
                       + [s6 for i in range(n6)] )
    config = [np.random.normal(0,1)+const7 for i in range(n7)] + [np.random.normal(0,1) for i in range(n1)] + [np.random.normal(0,1)+const2 for i in range(n2)] + [np.random.normal(0,1) for i in range(n3)] + [np.random.normal(0,1) for i in range(n4)] + [np.random.normal(0,1)+const5 for i in range(n5)] + [np.random.normal(0,1) for i in range(n6)]
    config = np.matrix([[1] if i>np.median(config) else [0] for i in config])
    sample = np.concatenate((config, overlap), axis=1)
    df = pd.DataFrame(sample, columns = ["phenotype"] + ["gene{}".format(i+1) for i in range(sample.shape[1]-1)])
    for col in df.columns:
        df[col] = df[col].astype(np.uint8)
    output = feather.write_dataframe(df, "data/sample.feather")
    return df

In [4]:
n000 = 600
n100 = 100
n010 = 100
n001 = 100
n110 = 100
n011 = 100
n111 = 100
diff100 = 1
diff110 = 0.66
diff111 = 0.33
sample = sample(n1=n000, n2=n100, n3=n010, n4=n001, n5=n110, n6=n011, n7=n111, 
                const7=diff111, const2=diff100, const5=diff110)

In [5]:
print (sample)

      phenotype  gene1  gene2  gene3
0             1      1      1      1
1             1      1      1      1
2             1      1      1      1
3             1      1      1      1
4             0      1      1      1
5             1      1      1      1
6             1      1      1      1
7             0      1      1      1
8             0      1      1      1
9             1      1      1      1
10            1      1      1      1
11            0      1      1      1
12            0      1      1      1
13            1      1      1      1
14            1      1      1      1
15            1      1      1      1
16            1      1      1      1
17            1      1      1      1
18            1      1      1      1
19            0      1      1      1
20            0      1      1      1
21            1      1      1      1
22            1      1      1      1
23            1      1      1      1
24            1      1      1      1
25            0      1      1      1
2

In [6]:
def get_pvalue(gene_df):
    cases = gene_df[gene_df["phenotype"]==1]
    ctrls = gene_df[gene_df["phenotype"]==0]
    pvalue_dict = {}
    for gene in list(gene_df)[1:]:
        n_gene_case = len(cases[cases[gene]==1])
        n_nogene_case = len(cases[cases[gene]==0])
        n_gene_ctrl = len(ctrls[ctrls[gene]==1])
        n_nogene_ctrl = len(ctrls[ctrls[gene]==0])
        p_value = pvalue(n_gene_case, n_gene_ctrl, n_nogene_case, n_nogene_ctrl).two_tail
        pvalue_dict[gene] = p_value
    return pvalue_dict

In [7]:
get_pvalue(sample)

{'gene1': 3.924994195915807e-06,
 'gene2': 0.016874767220460888,
 'gene3': 0.5485481780664612}